In [1]:
from shlink import Shlink
from datetime import datetime
import pandas as pd
from user_agents import parse

# Define credentials
shlink_url = 'https://s.stiv.tech/'
shlink_api_key = 'zRoykl8q6wOZW0TKhgIWPvyaCz7uh76U6uFBTYRT1XYPVv2dYFUzSWw9JKL0PE3i8sWz5mZwzFoxcyP8iNH5FceyXeDUQL4wSXs9gNL2bqXfAV2mrHog079yo1lzQHT7'

# Connecto to Shlink API
shlink = Shlink(url=shlink_url, api_key=shlink_api_key)

In [2]:
# Extract the list of short urls with the information
short_urls = shlink.list_short_urls()

# Extract the 'data' section
data_list = short_urls['shortUrls']['data']

# Create a DataFrame
urls_df = pd.DataFrame(data_list)

# Drop unwanted columns
unwanted_columns = ['deviceLongUrls', 'meta', 'domain', 'crawlable', 'forwardQuery', 'visitsCount']
urls_df = urls_df.drop(columns=unwanted_columns, errors='ignore')

# Separate 'visitsSummary' into individual columns
urls_df[['total', 'nonBots', 'bots']] = urls_df['visitsSummary'].apply(lambda x: pd.Series([x['total'], x['nonBots'], x['bots']]))

# Drop the original 'visitsSummary' column
urls_df = urls_df.drop(columns='visitsSummary')

urls_df

,shortCode,shortUrl,longUrl,dateCreated,tags,title,total,nonBots,bots
0,babypeso,https://s.stiv.tech/babypeso,https://cloud.stiv.tech/apps/forms/s/Msbfq8gqJ...,2024-01-20T03:37:08+00:00,[],None,0,0,0
1,baby,https://s.stiv.tech/baby,https://cloud.stiv.tech/apps/forms/s/Szy6Lrbmy...,2024-01-20T02:49:16+00:00,[victorcosas],None,24,24,0
2,dsa3P,https://s.stiv.tech/dsa3P,https://github.com/RisingTempest/shlink-py,2024-01-18T03:10:11+00:00,"[api, github, shlink, updated, working]",Shlink Github,3,3,0
3,OtIOM,https://s.stiv.tech/OtIOM,https://github.com/STIV-Cloud-Gaming-Hosting,2024-01-08T23:26:14+00:00,[],Prueba Discord,2,1,1
4,github,https://s.stiv.tech/github,https://github.com/STIV-Cloud-Gaming-Hosting,2024-01-02T19:47:14+00:00,[],Prueba con Github,13,10,3
5,U4VLj,https://s.stiv.tech/U4VLj,https://www.google.com,2023-12-27T16:40:04+00:00,[],Testing Shlink,39,38,1


In [4]:
# Create an empty DataFrame to store visit data
visits_info_df = pd.DataFrame()

# Loop through the urls_df and use each 'shortCode' in list_visit function
for index, row in urls_df.iterrows():
    shortCode = row['shortCode']
    visits_data = shlink.list_visit_data(shortCode)
    
    if 'visits' in visits_data and 'data' in visits_data['visits']:
        visits_data_list = visits_data['visits']['data']
        df_visits = pd.DataFrame(visits_data_list)
        df_visits['shortCode'] = shortCode  # Add shortCode as a column

        # Check if 'visitLocation' is in columns before normalizing
        if 'visitLocation' in df_visits.columns:
            visit_location_df = pd.json_normalize(df_visits['visitLocation'])
            df_visits = pd.concat([df_visits, visit_location_df], axis=1)
        
            # Drop the original 'visitLocation' column
            df_visits = df_visits.drop(columns='visitLocation')

        visits_info_df = pd.concat([visits_info_df, df_visits])

# Create new columns for browser, operating system, and device
visits_info_df['Browser'] = visits_info_df['userAgent'].apply(lambda x: parse(x).browser.family)
visits_info_df['Operating_System'] = visits_info_df['userAgent'].apply(lambda x: parse(x).os.family)
visits_info_df['Device'] = visits_info_df['userAgent'].apply(lambda x: parse(x).device.family)

# Drop the original 'userAgent' column
visits_info_df.drop(columns=['userAgent'], inplace=True)

# Display the modified DataFrame
visits_info_df

,shortCode,referer,date,potentialBot,countryCode,countryName,regionName,cityName,latitude,longitude,timezone,isEmpty,Browser,Operating_System,Device
0,baby,,2024-01-21T21:03:50+00:00,False,CL,Chile,Santiago Metropolitan,Santiago,-33.4521,-70.6536,America/Santiago,False,Samsung Internet,Android,Samsung SM-A217M
1,baby,,2024-01-21T21:03:34+00:00,False,CL,Chile,Santiago Metropolitan,Santiago,-33.4521,-70.6536,America/Santiago,False,Mobile Safari,iOS,iPhone
2,baby,,2024-01-21T21:03:15+00:00,False,CL,Chile,Santiago Metropolitan,Santiago,-33.4521,-70.6536,America/Santiago,False,Chrome Mobile,Android,K
3,baby,,2024-01-21T21:02:28+00:00,False,CL,Chile,Santiago Metropolitan,Santiago,-33.4521,-70.6536,America/Santiago,False,Chrome Mobile,Android,K
4,baby,,2024-01-21T21:01:39+00:00,False,CL,Chile,Santiago Metropolitan,Santiago,-33.4521,-70.6536,America/Santiago,False,Chrome Mobile,Android,K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34,U4VLj,,2023-12-27T19:17:49+00:00,False,,,,,0.0000,0.0000,,True,Chrome,Linux,Other
35,U4VLj,,2023-12-27T17:01:11+00:00,False,,,,,0.0000,0.0000,,True,Chrome Mobile,Android,K
36,U4VLj,,2023-12-27T16:40:36+00:00,False,,,,,0.0000,0.0000,,True,Chrome,Linux,Other
37,U4VLj,,2023-12-27T16:40:24+00:00,True,,,,,0.0000,0.0000,,True,WhatsApp,Other,Spider
